In [20]:
import numpy as np
import pandas as pd
import json
pd.set_option('display.max_columns', 130)

In [21]:

def openjson(d):
    #d is a dictionary which has been stored as a json file
    d=d["data"]#splitting the dictionary and accessing its data
    d=d["weather"]#splitting the dictionary and accessing its data
    j=d[0]["hourly"]#hourly is a json a file inside a json file , j=d[0]["hourly"] is used to access the first 24 hours of the day and its corresponding weather
    weatherdata=pd.DataFrame(j)#dataframe containing the weather data of the first day
    for i in range(1,len(d)):
        k=d[i]["hourly"]#iteratively stores the weather data of every day byour
        weatherdata_iterator=pd.DataFrame(k)
        weatherdata=pd.concat([weatherdata,weatherdata_iterator])#dataframe has the entire weather data by hour 
    a=np.arange(len(weatherdata))
    weatherdata.index=a#setting a uniform index for the dataframe
    data=pd.DataFrame(d)# dataframe that has information about this whole month  
    no_of_days=len(data)# no of days in this month
    o=d[0]["astronomy"]
    moondata=pd.DataFrame(o)
    for i in range(1,len(d)):
        p=d[i]["astronomy"]
        moondata_iterator=pd.DataFrame(p)
        moondata=pd.concat([moondata,moondata_iterator]) 
    moondata.index=np.arange(len(moondata))
    moondata["date"]=data["date"]
    data=pd.concat([data,moondata],axis=1)
    hour_day_conversion1=data.copy()
    hour_day_conversion2=pd.DataFrame()
    for i in range(0,no_of_days):
        for j in range(0,24):
            hour_day_conversion1.iloc[j,:]=data.iloc[i,:]
        hour_day_conversion1=hour_day_conversion1.iloc[0:24,:]    
        hour_day_conversion2=pd.concat([hour_day_conversion2,hour_day_conversion1])
    hour_day_conversion2.index=np.arange(len(hour_day_conversion2))
    final_data=pd.concat([hour_day_conversion2,weatherdata],axis=1)
    final_data.drop(["astronomy","hourly"],axis=1,inplace=True)
    return final_data




In [ ]:
c="C:\\Users\\saicharan\\Desktop\\project\\"
p="On_Time_On_Time_Performance_"
h=["2016","2017"]
l=["","ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO","MIA","ORD","PHX","SEA","SFO"]
maindf=pd.DataFrame()
for k in range(2):
    data_16_17=pd.DataFrame()
    for i in range(1,13):
        #data_16_17_iterator=pd.DataFrame()
        f=p+str(h[k])+"_"+str(i)+".csv"
        print(f)
        flight_data=pd.read_csv(f,low_memory=False)
        flight_data["time"]=flight_data.CRSDepTime/100
        flight_data.time=flight_data.time.astype("int")
        data_16_17_iterator=pd.DataFrame()
        for j in range(1,len(l)):
            e=c+str(l[j])+"\\"+str(h[k])+"-"+str(i)+".json"
            print(e)
            f=open(e,"r")
            d=json.load(f)
            weather_data=openjson(d)
            weather_data["loc"]=l[j]
            weather_data["time"]=weather_data.index%24
            weather_data.columns=['off', 'maxtempC', 'maxtempF', 'mintempC', 'mintempF', 'sunHour',
           'totalSnow_cm', 'uvIndex', 'moon_illumination', 'moon_phase',
           'moonrise', 'moonset', 'sunrise', 'sunset', 'date', 'DewPointC',
           'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC', 'HeatIndexF',
           'WindChillC', 'WindChillF', 'WindGustKmph', 'WindGustMiles',
           'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'tempF',
           'time', 'visibility', 'weatherCode', 'weatherDesc', 'weatherIconUrl',
           'winddir16Point', 'winddirDegree', 'windspeedKmph', 'windspeedMiles',
           'loc']
            weather_data.drop(["off"],axis=1,inplace=True)
            data_16_17_iterator=pd.concat([data_16_17_iterator,flight_data.merge(weather_data,how='inner',left_on=["Origin","time","FlightDate"],right_on=["loc","time","date"],validate="many_to_many")],axis=0)
            data_16_17_iterator=data_16_17_iterator[(data_16_17_iterator.Dest=="ATL") | (data_16_17_iterator.Dest=="CLT")| (data_16_17_iterator.Dest=="DEN")| (data_16_17_iterator.Dest=="DFW")| (data_16_17_iterator.Dest=="EWR")| (data_16_17_iterator.Dest=="IAH")| (data_16_17_iterator.Dest=="JFK")| (data_16_17_iterator.Dest=="LAS")| (data_16_17_iterator.Dest=="LAX")| (data_16_17_iterator.Dest=="MCO")| (data_16_17_iterator.Dest=="MIA")| (data_16_17_iterator.Dest=="ORD")| (data_16_17_iterator.Dest=="PHX")| (data_16_17_iterator.Dest=="SEA")| (data_16_17_iterator.Dest=="SFO")]
        data_16_17=pd.concat([data_16_17,data_16_17_iterator],axis=0)
    maindf=pd.concat([maindf,data_16_17],axis=0)   
maindf

In [ ]:
maindf=maindf[["FlightDate","Quarter","Year","Month","DayofMonth","DepTime","DepDel15","CRSDepTime","DepDelayMinutes","OriginAirportID","DestAirportID","ArrTime","CRSArrTime","ArrDel15","ArrDelayMinutes","windspeedKmph","winddirDegree","weatherCode","precipMM","visibility","pressure","cloudcover","DewPointF","WindGustKmph","tempF","WindChillF","humidity","date","time","loc"]]
maindf=maindf.dropna()
maindf=maindf.iloc[:,1:31]
maindf.drop_duplicates(inplace=True)
maindf.to_csv("maindataframe.csv")        


In [24]:
maindf

,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,loc
0,1,2016,1,1,808.0,0.0,810,0.0,10397,13303,1001.0,1016,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
1,1,2016,1,1,810.0,0.0,812,0.0,10397,11298,928.0,943,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
2,1,2016,1,1,852.0,0.0,850,2.0,10397,11057,957.0,1009,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
3,1,2016,1,1,900.0,0.0,855,5.0,10397,13303,1037.0,1045,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
4,1,2016,1,1,849.0,0.0,850,0.0,10397,11298,1001.0,1016,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
5,1,2016,1,1,821.0,0.0,825,0.0,10397,12478,1025.0,1037,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
8,1,2016,1,1,822.0,0.0,820,2.0,10397,12892,1014.0,1025,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
12,1,2016,1,1,850.0,0.0,845,5.0,10397,11292,1023.0,1040,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
21,1,2016,1,1,851.0,0.0,855,0.0,10397,13204,1007.0,1021,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
22,1,2016,1,1,815.0,0.0,810,5.0,10397,14771,1024.0,1031,0.0,0.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,8,ATL
